## metaclass的基本使用
> 什么是metaclass：  
- 继承了type的类就是Metaclass
- metaclass是用来创建类的类, 在实例化类的时候就会执行相关代码

**MetaClass可以应用于，根据参数动态的创建创建类的场景**，可参考后面的示例。

### 1. 用type创建一个类
> 为了方便理解metaclass，我们先尝试用type来创建个类

#### 1-1: 先用class创建类
> `type(name, bases, **kwargs)`：  
- `name`: 是类的名字
- `bases`: 是继承的基类
- `kwargs`: 字典类型的一些其它属性

**type相关的文档**，重点查看`__init__`方法：
```python
class type(object):
    """
    type(object_or_name, bases, dict)
    type(object) -> the object's type
    type(name, bases, dict) -> a new type
    """
    def mro(self, *args, **kwargs): # real signature unknown
        """ Return a type's method resolution order. """
        pass

    def __call__(self, *args, **kwargs): # real signature unknown
        """ Call self as a function. """
        pass

    def __delattr__(self, *args, **kwargs): # real signature unknown
        """ Implement delattr(self, name). """
        pass

    def __dir__(self, *args, **kwargs): # real signature unknown
        """ Specialized __dir__ implementation for types. """
        pass

    def __getattribute__(self, *args, **kwargs): # real signature unknown
        """ Return getattr(self, name). """
        pass

    def __init__(cls, what, bases=None, dict=None): # known special case of type.__init__
        """
        type(object_or_name, bases, dict)
        type(object) -> the object's type
        type(name, bases, dict) -> a new type
        # (copied from class doc)
        """
        pass
  ```

In [1]:
class People:
    
    def __init__(self, name, age):
        self.name = name
        self.age = age

In [2]:
p = People("codelieche", 20)
print(p.name, p.age)

codelieche 20


In [3]:
type(People)

type

#### 1-2: 我们使用type创建一个类

In [4]:
User = type("User", (), {"name": "codelieche", "age": 21})

In [5]:
User

__main__.User

In [6]:
type(User)

type

In [7]:
# 实例化User
u = User()

In [8]:
print(u.name, u.age)

codelieche 21


#### 1-3: 用type继承类

In [9]:
class Base:
    
    def say(self):
        print("I Can Say")

In [10]:
info = {"name": "codelieche", "age": 28}
User2 = type("User2", (Base,), info)

In [11]:
User2

__main__.User2

In [12]:
# 实例化对象
u2 = User2()

In [13]:
u2.say()

I Can Say


In [14]:
print(u2.name, u2.age)

codelieche 28


In [15]:
print(type(type), type.__mro__)

<class 'type'> (<class 'type'>, <class 'object'>)


> **发现：**`type`是`type`自身的实例，`type`又继承了`object`.

### 2. 元类示例

#### 2-1. 查看ABCMeta源码
> 查看源码，从源码中学习是一种比较好的方式。我们现在看下ABCMeta这个`metaclass`

In [16]:
# 引入类
from abc import ABCMeta

In [17]:
# ABCMeta??

```python
class ABCMeta(type):
        
        def __new__(mcls, name, bases, namespace, **kwargs):
            cls = super().__new__(mcls, name, bases, namespace, **kwargs)
            _abc_init(cls)
            return cls
 ```

#### 2-2. 编写个多级继承

In [18]:
# 编写个demo
class A:
    
    def __new__(cls, *args, **kwargs):
        print("A: __new__", cls, args, kwargs)
        return super().__new__(cls)
    
    
class B(A):
    
    def __new__(cls, *args, **kwargs):
        print("B: __new__", cls, args, kwargs)
        return super().__new__(cls, *args, **kwargs)
    
class C(B):
    
    def __new__(cls, *args, **kwargs):
        print("C: __new__", cls, args, kwargs)
        return super().__new__(cls, *args, **kwargs)
    
    def __init__(self, value=0):
        self.value = value
        

In [19]:
# 实例化
c1 = C(123)

C: __new__ <class '__main__.C'> (123,) {}
B: __new__ <class '__main__.C'> (123,) {}
A: __new__ <class '__main__.C'> (123,) {}


In [20]:
c1.value

123

#### 2-3. 设置metaclass

In [21]:
# 创建一个元类
class MyMetaClass(type):
    
    def __new__(mcls, name, bases, kwargs):
        print("MyMetaClass: __new__:")
        print("\t 1. 元类mcls:", mcls)
        print("\t 2. 类名称name:", name)
        print("\t 3. 继承的类bases:", bases)
        print("\t 4. 其它属性kwargs:", kwargs)
        
        # 参考type创建类需传递的参数
        return super().__new__(mcls, name, bases, kwargs)
    
    def __init__(self, *args, **kwargs):
        print("\n执行MyMetaClass.__init__：")
        print("\t args:", args)
        print("\t kwargs:", kwargs)
        return super().__init__(*args, **kwargs)

In [22]:
# 创建类C2，继承B，并且设置metaclass
class C2(B, metaclass=MyMetaClass):
    
    name = "Just for test"
    
    def __new__(cls, *args, **kwargs):
        print("C: __new__", cls, *args, kwargs)
        return super().__new__(cls)
    
    def __init__(self, value):
        self.value = value

MyMetaClass: __new__:
	 1. 元类mcls: <class '__main__.MyMetaClass'>
	 2. 类名称name: C2
	 3. 继承的类bases: (<class '__main__.B'>,)
	 4. 其它属性kwargs: {'__module__': '__main__', '__qualname__': 'C2', 'name': 'Just for test', '__new__': <function C2.__new__ at 0x1096f8d40>, '__init__': <function C2.__init__ at 0x1096f8c20>, '__classcell__': <cell at 0x1096feb10: empty>}

执行MyMetaClass.__init__：
	 args: ('C2', (<class '__main__.B'>,), {'__module__': '__main__', '__qualname__': 'C2', 'name': 'Just for test', '__new__': <function C2.__new__ at 0x1096f8d40>, '__init__': <function C2.__init__ at 0x1096f8c20>, '__classcell__': <cell at 0x1096feb10: MyMetaClass object at 0x7fc1f6555d10>})
	 kwargs: {}


> 在实例化C2类对象的时候，`MyMetaClass`中的`__new__`方法就被执行了。

In [23]:
c2 = C2("abc")

C: __new__ <class '__main__.C2'> abc {}
B: __new__ <class '__main__.C2'> () {}
A: __new__ <class '__main__.C2'> () {}


> 实例化对象的时候，所有非元类的`__new__`会执行，按照MRO顺序。

### 3. 动态创建类
> 我们创建一个类，如果设置了swim = True，那么其有swim()方法，如果设置了say = True，那么其有say()方法。

#### 3-1: 通过函数动态生成

In [24]:
def generate_cls(name, swim=False, say=False):
    
    def swim_func(self):
        print("我能游泳")
        
    def say_func(self):
        print("我能说话")
        
    kwargs = {}
    if swim:
        kwargs["swim"] = swim_func
        
    if say:
        kwargs["say"] = say_func
        
    # 生成类
    return type(name, (), kwargs)

In [25]:
# 创建一个能swim的类
Demo1 = generate_cls("Demo1", swim=True, say=False)

In [26]:
Demo1

__main__.Demo1

In [27]:
# 实例化
d1 = Demo1()

In [28]:
d1.swim()

我能游泳


In [29]:
hasattr(d1, "say")

False

In [30]:
# 创建一个能游泳和说话的类
Demo2 = generate_cls("Demo2", swim=True, say=True)

In [31]:
Demo2

__main__.Demo2

In [32]:
# 实例化
d2 = Demo2()

In [33]:
d2.say()

我能说话


In [34]:
d2.swim()

我能游泳


> **通过函数，我们就可以动态生成类了**，但是还是不够方便，这个时候就可以考虑用元类了。

#### 3-2: 通过元类动态创建类

- 创建元类

In [35]:
def swim_func(self):
        print("我能游泳")
        
def say_func(self):
    print("我能说话")

class DemoMetaClass(type):
    
    def __new__(mcls, name, bases, kwargs):
        
        if kwargs.get("swim"):
            kwargs["swim"] = swim_func
            
        if kwargs.get("say"):
            kwargs["say"] = say_func
        
        return super().__new__(mcls, name, bases, kwargs)
    

- 使用元类
> 创建个People的类，即会游泳，也会说话；创建个Duck的类，它只会游泳。

In [36]:
# 创建类People
class People(metaclass=DemoMetaClass):
    
    swim = True
    say = True
    
    def __init__(self, name, age):
        self.name = name
        self.age = age

In [37]:
# 查看People的方法
People.swim

<function __main__.swim_func(self)>

In [38]:
People.say

<function __main__.say_func(self)>

> **发现我们对类设置的是`swim = True`和`say = True`**，由于设置了元类，在实例化People类对象的时候，动态设置了方法。

In [39]:
# 创建类Duck
class Duck(metaclass=DemoMetaClass):
    
    swim = True
    say = False
    
    def __init__(self, name):
        self.name = name

In [40]:
# 查看Duck的方法
Duck.swim

<function __main__.swim_func(self)>

In [41]:
Duck.say

False

> **由于`say = False`，在实例化Duck类对象的时候，未给其赋值为方法。**  
到这里我们发现使用`metaclass`就可以很方便的动态生成类了。